In [ ]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt

MAX_X = 1e8
STEPS_RESOLUTION = int(1e6)

In [ ]:
## Read online experiments

df = pd.concat([pd.read_csv(f) for f in Path('online').glob('*.csv')])
df['env'] = df['env'].str.slice(6)  # drop dmlab_ prefix
df['version'] = df['run'].str.extract(r'((?:_[a-z]+[0-9]+)+)', expand=False).fillna('').str.lstrip('_')  # extract version
df = df[~df['env_steps'].isna()]
df['env_steps'] = (df['env_steps'].astype(int) // STEPS_RESOLUTION) * STEPS_RESOLUTION  # bucketize env_steps
df = df.groupby(['env', 'version', 'env_steps']).mean().reset_index()  # aggregate
data = df
# df['version'].unique()
df

In [ ]:
## Read baselines

df = pd.read_csv('baselines/dmlab_baselines.csv')
df = df[df['method'].isin(['r2d2_plus_1e10'])]
df['env'] = df['env'].str.slice(6)  # drop dmlab_ prefix
df = pd.concat([df.assign(env_steps=0), df.assign(env_steps=MAX_X)])
data_baselines = df

In [ ]:
## DMLab30 figure

df = data
dfb = data_baselines

dfg = df.groupby(['env'])
fig, axes = plt.subplots(nrows=6, ncols=5, figsize=(18,12), sharex=True)

for (key_env, g_env), ax in zip(dfg, axes.flatten()):
    # Plot experiment
    for key_version, g in g_env.groupby(['version']):
        ax.plot(g['env_steps'], g['return'], label=key_version)
    # Plot baselines
    dfbf = dfb[dfb['env'] == key_env]
    if len(dfbf) > 0:
        ax.plot(dfbf['env_steps'], dfbf['return'], color='gray', linestyle='dashed')  # label=dfbf.iloc[0]['method']
    # Chart properties
    ax.set_title(key_env)
    ax.set_xlim([0, MAX_X])
    ax.set_ylim(bottom=min(g_env['return'].min(), -5))

fig.legend()
plt.show()